## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df_clean = application_df.drop(columns=["EIN", "NAME"])

In [3]:
# Determine the number of unique values in each column.
for column in application_df_clean.columns:
    print(f"{column} has {application_df_clean[column].nunique()} unique values")

APPLICATION_TYPE has 17 unique values
AFFILIATION has 6 unique values
CLASSIFICATION has 71 unique values
USE_CASE has 5 unique values
ORGANIZATION has 4 unique values
STATUS has 2 unique values
INCOME_AMT has 9 unique values
SPECIAL_CONSIDERATIONS has 2 unique values
ASK_AMT has 8747 unique values
IS_SUCCESSFUL has 2 unique values


In [7]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df_clean["APPLICATION_TYPE"].value_counts()
print(application_counts)

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64


In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
cutoff = 500
application_types_to_replace = application_counts[application_counts < cutoff].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df_clean['APPLICATION_TYPE'] = application_df_clean['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df_clean['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df_clean["CLASSIFICATION"].value_counts()
print(classification_counts)

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64


In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts > 1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [15]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cutoff = 1000
classifications_to_replace = classification_counts[classification_counts < cutoff].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [16]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df_numeric = pd.get_dummies(application_df_clean, columns=["APPLICATION_TYPE",
                                                                       "AFFILIATION",
                                                                       "CLASSIFICATION",
                                                                       "USE_CASE",
                                                                       "ORGANIZATION",
                                                                       "INCOME_AMT",
                                                                       "SPECIAL_CONSIDERATIONS"])

In [19]:
# Split our preprocessed data into our features and target arrays
X = application_df_numeric.drop(columns=["IS_SUCCESSFUL"])
y = application_df_numeric["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=X_train_scaled.shape[1]))

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                8720      
                                                                 
 dense_1 (Dense)             (None, 80)                6480      
                                                                 
 dense_2 (Dense)             (None, 80)                6480      
                                                                 
 dense_3 (Dense)             (None, 1)                 81        
                                                                 
Total params: 21761 (85.00 KB)
Trainable params: 21761 (85.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
train_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 4s 3ms/step - loss: 0.5701 - accuracy: 0.7245
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5517 - accuracy: 0.7316
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5483 - accuracy: 0.7353
Epoch 4/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5456 - accuracy: 0.7348
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5449 - accuracy: 0.7363
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5449 - accuracy: 0.7359
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5433 - accuracy: 0.7359
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5428 - accuracy: 0.7357
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5418 - accuracy: 0.7364
Epoch 10/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5420 - accura

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5807 - accuracy: 0.7256 - 400ms/epoch - 1ms/step
Loss: 0.5807377696037292, Accuracy: 0.7255976796150208


In [28]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity.h5")

In [30]:
# Adjust the data
application_df_clean = application_df.drop(columns=['EIN', 'SPECIAL_CONSIDERATIONS'])

# Calculate value counts for the NAME column
name_counts = application_df['NAME'].value_counts()

# Determine which values to replace
names_to_replace = name_counts[name_counts < 5].index

# Replace in DataFrame
for name in names_to_replace:
    application_df['NAME'] = application_df['NAME'].replace(name, "Other")


In [31]:
# Adjust cutoff for binning
cutoff = 200
application_counts = application_df_clean['APPLICATION_TYPE'].value_counts()
application_types_to_replace = application_counts[application_counts < cutoff].index.tolist()

for app in application_types_to_replace:
    application_df_clean['APPLICATION_TYPE'] = application_df_clean['APPLICATION_TYPE'].replace(app, "Other")
